ABANDONED
=========

In [1]:
!pip install transformers
!pip install torch
!pip install numpy
!pip install random
!pip install pandas
!pip install sklearn

ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AlbertTokenizer, AlbertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

c:\Users\aresm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Carica il dataset
data_path = "../data/augmented_lyrics.csv"
df = pd.read_csv(data_path)

In [4]:
# Definizione del limite massimo di canzoni per artista
MAX_SONGS_PER_ARTIST = 10

# Applicazione dell'undersampling
# Raggruppa per artista e taglia randomicamente se il numero di canzoni supera il limite
undersampled_df = df.groupby('artist').apply(lambda x: x.sample(min(len(x), MAX_SONGS_PER_ARTIST))).reset_index(drop=True)

# Bilanciamento delle classi per genere
balanced_df = pd.DataFrame()
for genre in ['rock', 'pop', 'rap', 'rb', 'country']:
    genre_df = undersampled_df[undersampled_df['tag'] == genre]
    samples_per_genre = 20  # o il numero che desideri
    # Se ci sono meno campioni di quanto necessario, fai oversampling
    if len(genre_df) < samples_per_genre:
        genre_df = genre_df.sample(samples_per_genre, replace=True)
    else:
        genre_df = genre_df.sample(samples_per_genre)
    balanced_df = pd.concat([balanced_df, genre_df])

# Verifica il bilanciamento finale
print(balanced_df['tag'].value_counts())
balanced_df["artist"].value_counts()

tag
rock       20
pop        20
rap        20
rb         20
country    20
Name: count, dtype: int64


artist
Circa Waves       1
Kimbra            1
Jon Vinyl         1
Kirk Franklin     1
Asiahn            1
                 ..
The Chameleons    1
Nina              1
Solarsuit         1
Fog Lake          1
Shania Twain      1
Name: count, Length: 100, dtype: int64

In [5]:
df = balanced_df

In [6]:
print(df['tag'].value_counts())


tag
rock       20
pop        20
rap        20
rb         20
country    20
Name: count, dtype: int64


In [7]:
# Converti le etichette di testo in numeri
label_encoder = LabelEncoder()
df['tag'] = label_encoder.fit_transform(df['tag'])
labels = list(label_encoder.classes_)


In [8]:
df["artist"].value_counts()

artist
Circa Waves       1
Kimbra            1
Jon Vinyl         1
Kirk Franklin     1
Asiahn            1
                 ..
The Chameleons    1
Nina              1
Solarsuit         1
Fog Lake          1
Shania Twain      1
Name: count, Length: 100, dtype: int64

In [9]:

# Preparazione del dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)


In [10]:

# Converti DataFrame in Dataset di Hugging Face
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [11]:

# Tokenizzazione e aggiunta delle etichette
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

def tokenize_and_encode(data):
    encoding = tokenizer(data['lyrics'], padding="max_length", truncation=True, max_length=512)
    encoding['labels'] = torch.tensor(data['tag'])
    return encoding


In [12]:
# Preprocessa i datasets di allenamento, validazione e test: tokenizza e codifica i testi in batch
train_dataset = train_dataset.map(tokenize_and_encode, batched=True)
val_dataset = val_dataset.map(tokenize_and_encode, batched=True)
test_dataset = test_dataset.map(tokenize_and_encode, batched=True)


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map: 100%|██████████| 20/20 [00:00<00:00, 315.27 examples/s]


In [13]:

# Definizione del modello
model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=len(labels))


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [15]:

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer con compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)


c:\Users\aresm\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# Train the model
trainer.train()

 33%|███▎      | 9/27 [04:07<07:40, 25.56s/it]c:\Users\aresm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
                                              
 33%|███▎      | 9/27 [04:20<07:40, 25.56s/it]

{'eval_loss': 1.6029479503631592, 'eval_accuracy': 0.125, 'eval_f1': 0.0625, 'eval_precision': 0.041666666666666664, 'eval_recall': 0.125, 'eval_runtime': 12.1224, 'eval_samples_per_second': 0.66, 'eval_steps_per_second': 0.082, 'epoch': 1.0}


 67%|██████▋   | 18/27 [08:09<03:51, 25.76s/it]c:\Users\aresm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
                                               
 67%|██████▋   | 18/27 [08:21<03:51, 25.76s/it]

{'eval_loss': 1.648627758026123, 'eval_accuracy': 0.125, 'eval_f1': 0.0625, 'eval_precision': 0.041666666666666664, 'eval_recall': 0.125, 'eval_runtime': 11.5365, 'eval_samples_per_second': 0.693, 'eval_steps_per_second': 0.087, 'epoch': 2.0}


100%|██████████| 27/27 [12:06<00:00, 25.84s/it]c:\Users\aresm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
                                               
100%|██████████| 27/27 [12:17<00:00, 27.31s/it]

{'eval_loss': 1.6420588493347168, 'eval_accuracy': 0.25, 'eval_f1': 0.25, 'eval_precision': 0.4166666666666667, 'eval_recall': 0.25, 'eval_runtime': 11.0417, 'eval_samples_per_second': 0.725, 'eval_steps_per_second': 0.091, 'epoch': 3.0}
{'train_runtime': 737.3669, 'train_samples_per_second': 0.293, 'train_steps_per_second': 0.037, 'train_loss': 1.4716477570710358, 'epoch': 3.0}


TrainOutput(global_step=27, training_loss=1.4716477570710358, metrics={'train_runtime': 737.3669, 'train_samples_per_second': 0.293, 'train_steps_per_second': 0.037, 'total_flos': 5163520794624.0, 'train_loss': 1.4716477570710358, 'epoch': 3.0})

In [17]:
# Evaluate the model on the test dataset
results = trainer.evaluate(test_dataset)


100%|██████████| 3/3 [00:18<00:00,  6.25s/it]c:\Users\aresm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|██████████| 3/3 [00:18<00:00,  6.32s/it]

{'eval_loss': 1.4928160905838013, 'eval_accuracy': 0.4, 'eval_f1': 0.36214285714285716, 'eval_precision': 0.5075757575757576, 'eval_recall': 0.4, 'eval_runtime': 30.728, 'eval_samples_per_second': 0.651, 'eval_steps_per_second': 0.098, 'epoch': 3.0}


In [18]:

print(results)

{'eval_loss': 1.4928160905838013, 'eval_accuracy': 0.4, 'eval_f1': 0.36214285714285716, 'eval_precision': 0.5075757575757576, 'eval_recall': 0.4, 'eval_runtime': 30.728, 'eval_samples_per_second': 0.651, 'eval_steps_per_second': 0.098, 'epoch': 3.0}
